In [44]:
import tensorflow as tf
import numpy as np
import pickle

In [45]:
# Loading training image dataset
trainset_features = np.load('train_images.pkl')
train_labels = np.load('train_labels.pkl')
test_features = np.load('test_images.pkl')

# Normalizing the image data
train_features = (trainset_features - np.min(trainset_features))/np.max(trainset_features)
test_features = (test_features - np.min(test_features))/np.max(test_features)

print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)

(6034, 50, 50, 3)
(6034, 63)
(43377, 50, 50, 3)


In [46]:
def conv_2d(x,W): 
    # Padding : surround the original image with 0's 
    # valid padding - no padding.size of the image shrinks  resultant image : (n - f + 1, n - f + 1)
    # same padding - resultant is same as that of original. resultant image : (n + 2p - f + 1, n +2p - f + 1)  p = (f - 1)/2
    # Strides : ((n + 2p - f)/s + 1, (n +2p - f)/s + 1)
    return tf.nn.conv2d(input = x, filter = W, strides = [1,1,1,1], padding="SAME")

def avg_pool_2x2(x):
    # MAX/AVG Pooling : ((n + 2p - f)/s + 1, (n +2p - f)/s + 1)
    return tf.nn.avg_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


# Parameters and number of filters are the inspirations of AlexNet with little modifications
def conv2_network(x_image): 
    
    # First Conv and Pooling layers
    W_conv1 = tf.Variable(tf.truncated_normal([5,5,3,8], stddev=0.02))
    b_conv1 = tf.Variable(tf.zeros([8]))
    h_conv1 = tf.nn.relu(conv_2d(x_image, W_conv1) + b_conv1)
    h_pool1 = avg_pool_2x2(h_conv1)
    conv1 = tf.nn.dropout(h_pool1, keep_prob)

    # Seond Conv and Pooling layers
    W_conv2 = tf.Variable(tf.truncated_normal([5,5,8,32], stddev=0.02)) 
    b_conv2 = tf.Variable(tf.zeros([32]))
    h_conv2 = tf.nn.relu(conv_2d(conv1, W_conv2) + b_conv2)
    h_pool2 = avg_pool_2x2(h_conv2)
    conv2 = tf.nn.dropout(h_pool2, keep_prob)

    #Flattenning the output of ConvNets 
    flat_conv2 = tf.contrib.layers.flatten(conv2)

    # First fully connected layers
    datasize = flat_conv2.get_shape().as_list()[1]
    W_fc1 = tf.Variable(tf.truncated_normal([datasize, 64], stddev=0.02))
    b_fc1 = tf.Variable(tf.zeros([64]))
    h_fc1 = tf.nn.relu(tf.matmul(flat_conv2,W_fc1)+b_fc1)
    h_fc1 = tf.nn.dropout(h_fc1, keep_prob)

    # Output layer
    y_conv = tf.layers.dense(inputs=h_fc1, units=63)
    return y_conv

def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    out_batches = []
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batches = [features[start_i:end_i], labels[start_i:end_i]] 
        out_batches.append(batches)
    return out_batches

In [47]:
n_channels = 3 
n_classes = 63
im_size = 50

# Features and Labels
features = tf.placeholder(tf.float32, [None, im_size, im_size, n_channels])
labels = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(dtype=tf.float32, name = "keep_prob")

# Learning rate 
learning_rate = 0.01

# Output of the model 
output = conv2_network(features)

# Calculating cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = output, labels = labels))
# Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Train or test with this batch size
batchSize =  128 
epochs = 100


init = tf.global_variables_initializer() 

with tf.Session() as sess:
    sess.run(init) 
    train_batches = batches(batchSize, train_features, train_labels)
    
    for epoch in range(epochs): 
        for batch_features, batch_labels in train_batches: 
            train_data = {features: batch_features, labels : batch_labels, keep_prob : 0.7}
            training_accuracy = sess.run(accuracy, feed_dict = train_data )
            # Print status for every 10 epochs
        if epoch % 10 == 0:
            print('Epoch {:<3} - Training Accuracy: {}'.format(epoch,training_accuracy))

    # Save the variables to disk.
    save_model_path = './image_classification'
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)
    print("Model saved")

Epoch 0   - Training Accuracy: 0.0
Epoch 10  - Training Accuracy: 0.0
Epoch 20  - Training Accuracy: 0.0
Epoch 30  - Training Accuracy: 0.0555555559694767
Epoch 40  - Training Accuracy: 0.0
Epoch 50  - Training Accuracy: 0.0555555559694767
Epoch 60  - Training Accuracy: 0.0555555559694767
Epoch 70  - Training Accuracy: 0.0555555559694767
Epoch 80  - Training Accuracy: 0.0
Epoch 90  - Training Accuracy: 0.0
Model saved
